# Dust Collector Control System

Welcome to the *vade mecum* for the dust collection control system. This notebook documents the design, logic, and hardware/software integration for the project.

It’s written to be clear, practical, and useful for future reference or modification.  Why?

> “Memories are messages from past-you to present-you.”  
> — Mike Levin
> 
> If Mike were here, he'd probably grin and say,
> 
> "Make the messages clear. You never know how dumb future-you might be."
> 😄 *(quoted via Bob, in a moment of clarity and wire fumes)*



# How I Got Here

The project is an outgrowth of a major rework of the woodshop to improve usability, tool storage, and install a state-of-the-art dust collection system to replace the 25-year-old felt-bag setup.

I began by stripping the walls of the (stupid and ugly) pegboard mess I’d created, and installing a pleasant-looking French cleat system on three walls. This lets me keep things off the floor and stored in their own special places so I know where to find them, and where to put them back.

Along the way, I decided to add some automation. I had previously wired the blast gates so that opening a (homemade) gate would trip a microswitch, turning on a solid-state relay and starting the dust collector motor. When the gate closed, the microswitch disengaged and the motor shut off. Not having to walk over and flip a switch was a major convenience.

With the new dust collector, I wanted even more automation. The plan: use a split-core transformer around one of the wires leading to each machine (table saw, drill press, lathe, etc.) to detect when it turned on, then automatically open the corresponding blast gate and start the dust collector motor. When the machine shut off, the gate would close and the collector would turn off a few seconds later to flush the duct.

I started buying parts—plastic dust gates with rotating 'wipers', linear actuators, current sensors, stepper motors and controllers. But I decided to build my own blast gates again: I was upgrading from 4″ to 5″ ducts, and the commercial options seemed leaky, flimsy, or awkward to mount actuators on.

I chose a Raspberry Pi 4B as the brains of the system. Sure, it could be done with less—but I have big plans: switches, LEDs, monitors, temperature and air quality sensors, sound effects—whatever amuses me.

I also named my OpenAI ChatGPT account “Bob,” in honor of a brilliant and dear friend who passed away recently. Bob and I are collaborating on this project. For that reason, I can’t take credit for most of the elegant code herein—I just *told him to do it!*


# The Top Level of the System



## Python --  Why?
The language of choice is Python -- a language of deep and subtle beauty with a wealth of useful libraries for
embedded systems.  I can also use VSCode remotely from other machines to run and debug code on the Pi.  Oh brave
new world, that has such things in it.


## Directory Structure
```
DustCollector/
├── core/                ← 🟢 Shared abstractions go here
│   └── actuator.py      ← Defines Actuator base class
├── hardware/
│   └── real_actuator.py ← Inherits from Actuator
├── mock/
│   └── fake_actuator.py ← Inherits from Actuator
├── notebooks/
├── main.py
```

```
DustCollector/
├── code/                  # Top-level source code
│   ├── anaconda_projects/ # Artifact of Jupyter/Anaconda install
│   │   └── db/            # (Unused system metadata)
│   ├── hardware/          # Hardware-specific code (GPIO, sensors, etc.)
│   ├── images/            # Source diagrams, photos, and figures
│   ├── mock/              # Mock implementations for testing/dev
│   ├── notebooks/         # This Jupyter-based design document
│   ├── __pycache__/       # Auto-generated Python bytecode
│   └── test/              # Test scripts used during bring-up
└── docs/                  # Other docs (PDFs, notes, exports)
```

## Coding Style

* PEP8-compliant
* black -l 79 compliant
* flake8 compliant
* Modules: snake_case
* Classes: CamelCase
* Methods & variables: snake_case

### Gate Control State Machine

This diagram shows the states a gate can go through based on machine activity, motor state, and timer events.

![Gate state diagram](../images/gate-states.jpg)

This is a Moore-style state machine, meaning that the system’s outputs depend entirely on its current state—not directly on the inputs. Inputs trigger state transitions, and each state determines the system’s behavior. The interesting part happens in the **transitions**, which define how and when the system moves from one state to another.


Each transition is labeled with the event that triggers it, followed by a separator line and a summary of the actions taken *before* entering the new state. For clarity, these actions are usually wrapped as function calls rather than coded inline.

The trickiest part is managing multiple gates in a way that mimics intelligent human behavior. For example: if two gates are open, the dust collector shouldn't turn off until **both** are closed.

Each state checks specific conditions and responds accordingly. Note that the **timer** here is a singleton shared by all gates—it gets reset by *any* gate. This allows the system to keep the motor running as long as at least one gate is open, regardless of the order in which gates are opened or closed.

Bob has been instrumental in showing how to implement this sort of thing using Python's asyncio package.



## Hardware Overview

- **Raspberry Pi 4B**: main controller
- Multiple instances of the **MCP23017** digital I/O expander: for digital I/O (switches, LEDs, stepper control)
- Multiple **DRV8825 / TMC2209**: stepper drivers
- Multiple **Stepper motors**: open and close blast gates
- Multiple **Limit switches** (two per gate): detect gate open/closed
- Multiple **General purpose switches** to control things like auto/manual operation of gates, and whatever else I think of
- Multiple **LEDs** for indicators, flashy thingies, etc.
- Multiple **Current sensors** (one for each machine with a blast gate): detect machine activity
- Multiple **Display** -- some simple LCD, some 7-segment, some monitors, for whatever I want to do with them
- Multiple **Temperature sensors** (hell, why not?)
- A very capable Air Quality Monitor to determine how much particulate matter of given sizes is in the atmosphere so I can turn on a fan for the [Corsi-Rosenthal air filter](https://cleanaircrew.org/box-fan-filters/), and make sure it stays on long enough to clear the air.
- **5V and 12V power supplies** (to be mounted _outside_ the enclosure)  5V is for logic and Raspberry Pi, 12V for actuators

Wiring and terminal blocks are used to route signals cleanly into the controller enclosure.  Circuits to be mounted on acrylic boards with standoffs as needed.  My goal is to have something that
looks more or less like an adult did it, and not some high-school kid. That means a nice enclosure, nicely mounted circuit boards,
nicely routed wiring, connectors to the outside world, etc.

[link text](https://example.com)


## Software Architecture

The system is composed of:
- A high level controller that manages system operation
- `Actuator` objects, which control each gate (note -- Actuators can be either linear or stepper motors)
- Event detection logic from current sensors
- Async control loop that monitors events and runs state transitions

There are still details to be worked out regarding manual/auto control, such as:
- is each individual gate controllable manually, or does manual mode apply to all?
- what switches for manual mode -- momentary contact, dpdt, etc.

Look for a # Manual/Auto Mode section below


Each gate runs independently, responding to machine activity or manual control.

## Class Structure

We'll start with the actuators that actually move things.  An `Actuator` is an abstract base class that supports basic 'open' and 'close' operations.  From this are derived more specific types of actuators.  For example, a simple linear actuator just needs a few seconds of
12V applied to it to open or close, and a couple of limit switches to know when to stop. \[Most of these have limit switches internally, but I can't seem to find one that has exactly the travel I need, so ...\]

A slightly more complicated actuator uses a stepper motor to drive a rack-and-pinion system to move the gate, plus the usual limit switches.  This requires considerably more GPIO pins than the Pi can support, hence the multiplicity of MCP23017 expanders.

I suppose other types of actuators could be used -- I've seen Youtube videos of people using pneumatic actuators and so on.

In any case, the interface is simple -- open() or close(), as shown in the code below:

In [78]:
# Example: base class stub for an actuator
class Actuator:
    def __init__(self, name):
        self.name = name
        self.mode = 'auto'  # or 'manual'
        self.state = 'closed'

    def open(self):
        print(f"Opening gate {self.name}")
        self.state = 'open'

    def close(self):
        print(f"Closing gate {self.name}")
        self.state = 'closed'



## Notes

- Details of how to manage auto/manual mode are TBD.
- Manual control uses switches wired to GPIO via MCP23017.
- LED indicators reflect mode (green for auto, red for manual).

We’ll flesh this out in later sections, including event loop logic and integration with actual GPIO libraries.

Next, we can look at the StepperActuator class that actually drives the stepper motor, checks the limit switches and handles timeouts in the event that something is stuck.

In [106]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from code.actuator import Actuator
import time


class StepperController(Actuator):
    """Stepper motor actuator using DRV8825/TMC2209 + MCP23017."""

    def __init__(self, id, name, mcp, dir_pin, step_pin, en_pin,
                 limit_switch_open_id, limit_switch_closed_id,
                 switch_controller, steps_per_action=400, delay=0.002):
        super().__init__(id, name)
        self.mcp = mcp
        self.dir_pin = self.mcp.get_pin(dir_pin)
        self.step_pin = self.mcp.get_pin(step_pin)
        self.en_pin = self.mcp.get_pin(en_pin)
        self.switch_controller = switch_controller
        self.limit_switch_open_id = limit_switch_open_id
        self.limit_switch_closed_id = limit_switch_closed_id
        self.steps_per_action = steps_per_action
        self.delay = delay
        self.position = "unknown"

        self.dir_pin.direction = True
        self.step_pin.direction = True
        self.en_pin.direction = True
        self.disable()
        self.home()

    def enable(self):
        self.en_pin.value = False

    def disable(self):
        self.en_pin.value = True

    def move(self, direction, target_switch_id):
        self.dir_pin.value = direction
        self.enable()
        step_counter = 0
        while step_counter < self.steps_per_action:
            if self.switch_controller.read_switch(target_switch_id):
                print(f"{self.name} limit switch {target_switch_id} hit.")
                break
            self.step_pin.value = True
            time.sleep(self.delay)
            self.step_pin.value = False
            time.sleep(self.delay)
            step_counter += 1
        self.disable()

    def open(self):
        print(f"{self.name} opening...")
        self.move(direction=1, target_switch_id=self.limit_switch_open_id)
        self.position = "open"

    def close(self):
        print(f"{self.name} closing...")
        self.move(direction=0, target_switch_id=self.limit_switch_closed_id)
        self.position = "closed"

ModuleNotFoundError: No module named 'code.actuator'; 'code' is not a package